In [ ]:
# Install flory
!pip install flory


In [ ]:
import numpy as np
DTYPE = np.float64
import flory
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib widget
import os
import sys
import pandas as pd
from tqdm import tqdm
import plotly.graph_objects as go


In [ ]:
def floryHuggins(phi:DTYPE, chi:np.array):
    part_1 = np.sum(phi*np.log(phi))
    part_2 = 0

    for i in range(len(phi)):
        for j in range(i+1, len(phi)):
            part_2 += chi[i][j]*phi[i]*phi[j]

    return part_1 + part_2

### Comparing for a 2 component, 2 compartment case

In [ ]:
chis = [[0, 4], [4, 0]]
phi_means = [0.3, 0.7]

phases = flory.find_coexisting_phases(2, chis, phi_means)


print("\n" + f"Equlibrium phase volumes: {phases.volumes}")
print("Equlibrium phase concentrations:\n" + f"{phases.fractions}")


# 2d example
phi = np.linspace(0, 1, 100)
ones = np.ones(len(phi))
chi = chis[0][1]

F = phi*np.log(phi) + (ones-phi)*np.log(ones-phi) + chi*phi*(ones-phi)

plt.plot(phi, F)
plt.plot(ones-phi, F, "--")
plt.axvline(phi_means[0])
plt.axvline(phases.fractions[0, 0], label = f"{phases.fractions[0, 0]:.2f},{phases.fractions[0, 1]:.2f}", color  = "k")
plt.axvline(phases.fractions[0, 1], label = f"{phases.fractions[1, 0]:.2f},{phases.fractions[1, 1]:.2f}")
plt.legend(loc = "best")


In [ ]:
chis = [[0, 4.0], [4.0, 0]]
phi_global = [0.5, 0.5]

phases = flory.find_coexisting_phases(2, chis, phi_global, progress=False)

print("\n" + f"Equlibrium phase volumes: {phases.volumes}")
print("Equlibrium phase concentrations:\n" + f"{phases.fractions}")

In [ ]:
step_size = DTYPE(0.01)
n_points = 1000

eta_1_perturbed = np.linspace(phases.volumes[0] - n_points*step_size, phases.volumes[0]+n_points*step_size, 2*n_points+1)
phi_11_perturbed = np.linspace(phases.fractions[0, 0] - n_points*step_size, phases.fractions[0, 0]+n_points*step_size, 2*n_points+1)

accepted_eta_1 = []
accepted_phi_11 = []
accepted_phi_12 = []
accepted_F = []

for eta_1 in tqdm(eta_1_perturbed):
    if eta_1 > 1 or eta_1 < 0:
        continue
    
    eta_2 = 1 - eta_1
    
    for phi_11 in phi_11_perturbed:
        if phi_11 > 1 or phi_11 < 0:
            continue

        phi_12 = (phi_global[0] - eta_1*phi_11)/(1-eta_1)

        if phi_12 > 0 and phi_12 < 1:
            phi_21 = 1 - phi_11 
            phi_22 = 1 - phi_12
            
            phi_in_k1 = [phi_11, phi_21]
            phi_in_k2 = [phi_12, phi_22]
            
            F = 0
            F += eta_1*floryHuggins(phi_in_k1, chis)
            F += eta_2*floryHuggins(phi_in_k2, chis)
            
            accepted_eta_1.append(eta_1)
            
            accepted_phi_11.append(phi_11)
            accepted_phi_12.append(phi_12)
            
            accepted_F.append(F)

In [ ]:
df = pd.DataFrame()
df["eta_1"] = accepted_eta_1
df["phi_11"] = accepted_phi_11
df["phi_12"] = accepted_phi_12
df["F"] = accepted_F

df

In [ ]:
F = 0
for i in range(len(phases.volumes)):
    F += phases.volumes[i]*floryHuggins(phases.fractions[i, :], chis)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")

# Plot the data
ax.scatter(df["phi_11"], df["eta_1"], df["F"], c=df["F"], cmap="viridis", marker="o", alpha = 0.1)


ax.scatter(phases.fractions[0,0], phases.volumes[0], F, color = "k")

### 3 component case

In [ ]:
# Interaction matrix
chis = np.array([[0, 3.0, 3.0], [3.0, 0.0, 3.0], [3.0, 3.0, 0.0]], dtype = DTYPE)

# Global value of phis
phi_global = np.array([0.4, 0.3, 0.3], dtype = DTYPE)

# Equilibrium 3 phase solution
phases = flory.find_coexisting_phases(3, chis, phi_global, progress=False)

print("\n" + f"Equlibrium phase volumes: {phases.volumes}")
print("Equlibrium phase concentrations:\n" + f"{phases.fractions}")

In [ ]:
step_size = DTYPE(0.01)
n_points = 10

In [ ]:
# perturbing the phase volumes around the phase-separated volumes
eta_1_perturbed = np.linspace(phases.volumes[0] - n_points*step_size, phases.volumes[0]+n_points*step_size, 2*n_points+1)
eta_2_perturbed = np.linspace(phases.volumes[1] - n_points*step_size, phases.volumes[1]+n_points*step_size, 2*n_points+1)

# perturbing the concentrations around the phase separated ones
# notation: phi_ij -> phi for componenent i in compartment j
phi_11_perturbed = np.linspace(phases.fractions[0, 0]-n_points*step_size, phases.fractions[0, 0]+n_points*step_size, 2*n_points+1)
phi_12_perturbed = np.linspace(phases.fractions[0, 1]-n_points*step_size, phases.fractions[0, 1]+n_points*step_size, 2*n_points+1)
phi_21_perturbed = np.linspace(phases.fractions[1, 0]-n_points*step_size, phases.fractions[1, 0]+n_points*step_size, 2*n_points+1)
phi_22_perturbed = np.linspace(phases.fractions[1, 1]-n_points*step_size, phases.fractions[1, 1]+n_points*step_size, 2*n_points+1)


accepted_eta_1_perturbed = []
accepted_eta_2_perturbed = []

accepted_phi_11_perturbed = []
accepted_phi_12_perturbed = []
accepted_phi_21_perturbed = []
accepted_phi_22_perturbed = []

accepted_Fs = []

In [ ]:
ctr = 0 # Number of perturbations accepted

for eta_1 in tqdm(eta_1_perturbed):
    for eta_2 in eta_2_perturbed:
        if eta_1<0 or eta_2<0 or eta_1>1 or eta_2>1: # volume constraints
            continue

        eta_3 = 1 - eta_1 - eta_2 # Calculate the volume of the third compartment/phase
        if eta_3<0 or eta_3>1:
            continue
            
        for phi_11 in phi_11_perturbed:
            for phi_12 in phi_12_perturbed:
                if phi_11<0 or phi_11>1 or phi_12<0 or phi_12>1:
                    continue

                # Calculate the concentration of component 1 in compartment 3
                phi_13 = (phi_global[0] - eta_1*phi_11 - eta_2*phi_12)/(1-eta_1-eta_2)

                if phi_13 < 0 or phi_13 > 1:
                    continue

                for phi_21 in phi_21_perturbed:
                    for phi_22 in phi_22_perturbed:
                        if phi_21<0 or phi_21>1 or phi_22<0 or phi_22>1:
                            continue

                        # Calculate the concentration of componenent 2 in compartment 3
                        phi_23 = (phi_global[1] - eta_1*phi_21 - eta_2*phi_22)/(1-eta_1-eta_2)

                        if phi_23<0 or phi_23>1:
                            continue
                        ctr += 1


                        # Now the system is determined completely
                        phi_31 = 1 - phi_11 - phi_21
                        phi_32 = 1 - phi_12 - phi_22
                        phi_33 = 1 - phi_13 - phi_23

                        phi_in_k1 = [phi_11, phi_21, phi_31]
                        phi_in_k2 = [phi_12, phi_22, phi_32]
                        phi_in_k3 = [phi_13, phi_23, phi_33]


                        # print(eta_1*phi_13 + eta_2*phi_23 + eta_3*phi_33) # Should sum up to phi_global[2]
                        # if ctr%100000:
                        #     print(phi_in_k1)
                        #     print(phi_in_k2)
                        #     print(phi_in_k3)
                        #     print()

                        
                        # Calculate free energy
                        F = 0
                        
                        F += eta_1 * floryHuggins(phi_in_k1, chis)
                        F += eta_2 * floryHuggins(phi_in_k2, chis)
                        F += eta_3 * floryHuggins(phi_in_k3, chis)

                        accepted_Fs.append(F)
                        
                        accepted_eta_1_perturbed.append(eta_1)
                        accepted_eta_2_perturbed.append(eta_2)
                        
                        accepted_phi_11_perturbed.append(phi_11)
                        accepted_phi_12_perturbed.append(phi_12)
                        accepted_phi_21_perturbed.append(phi_21)
                        accepted_phi_22_perturbed.append(phi_22)

In [ ]:
df = pd.DataFrame()
df["eta_1"] = accepted_eta_1_perturbed
df["eta_2"] = accepted_eta_2_perturbed
df["phi_11"] = accepted_phi_11_perturbed
df["phi_12"] = accepted_phi_12_perturbed
df["phi_21"] = accepted_phi_21_perturbed
df["phi_22"] = accepted_phi_22_perturbed
df["F"] = accepted_Fs

df

In [ ]:
# Minimum value from the flory package
F_flory = 0
for i in range(len(phases.volumes)):
    F_flory += phases.volumes[i]*floryHuggins(phases.fractions[i, :], chis)

# Minimum value after perturbations
F_perturb = np.min(df["F"])

print(F_flory > F_perturb)

In [ ]:
min_F_row = df[df['F'] == df['F'].min()]
min_F_row

### Merging compartments

In [ ]:
# Interaction matrix
chis = np.array([[0, 3.0, 3.0], [3.0, 0.0, 3.0], [3.0, 3.0, 0.0]], dtype = DTYPE)

# Global value of phis
phi_global = np.array([0.333, 0.334, 0.333], dtype = DTYPE)

# Equilibrium 3 phase solution
phases = flory.find_coexisting_phases(3, chis, phi_global, progress=False)

print("\n" + f"Equlibrium phase volumes: {phases.volumes}")
print("Equlibrium phase concentrations:\n" + f"{phases.fractions}")

In [ ]:
# Take the transpose as the notation is different:
# For me: phi_ij = component i in compartment j
# For flory: phi_ji = component i in compartment j (OR I THINK SO!)

phases.fractions = np.transpose(phases.fractions)

In [ ]:
phases.fractions

In [ ]:
phases.volumes

In [ ]:
# Free energy of the 3 phase system

F_flory = 0
for i in range(len(phases.volumes)):
    F_flory += phases.volumes[i]*floryHuggins(phases.fractions[i, :], chis)

print(F_flory)


#### Merging compartments 1 and 2

In [ ]:
# Merging compartments 1 and 2

# First, calculate the modified compositions in the hybrid compartment (1+2)
eta_mod = phases.volumes[0] + phases.volumes[1]
phi_1mod = (phases.volumes[0]*phases.fractions[0, 0] + phases.volumes[1]*phases.fractions[0, 1])/eta_mod
phi_2mod = (phases.volumes[0]*phases.fractions[1, 0] + phases.volumes[1]*phases.fractions[1, 1])/eta_mod
phi_3mod = 1 - phi_1mod - phi_2mod

# So, now we have two phases with [phi_1mod, phi_2mod, phi_3mod] and [phases.fractions[0, 2], phases.fractions[1, 2], phases.fractions[2, 2]]
# with volumes (phases.volumes[0] + phases.volumes[1]) and phases.volumes[2] respectively

# For perturbations, we keep the volumes of these individual phases fixed
# Essentially, only perturb the concentrations

phi_in_kmod = [phi_1mod, phi_2mod, phi_3mod]
phi_in_k3 = [phases.fractions[0, 2], phases.fractions[1, 2], phases.fractions[2, 2]]

# Calculate the Free Energy of this modified system
F_mod = eta_mod*floryHuggins(phi_in_kmod, chis) + phases.volumes[2]*floryHuggins(phi_in_k3, chis)
print(F_mod)

In [ ]:
phi_1mod, phi_2mod

In [ ]:
np.min(phi_1mod_perturbed), np.max(phi_1mod_perturbed)

In [ ]:
# Effectively 2 degrees of freedom in the system
step_size = DTYPE(0.001)
n_points = 1000

phi_1mod_perturbed = np.linspace(phi_1mod-n_points*step_size, phi_1mod+n_points*step_size, 2*n_points+1)
phi_2mod_perturbed = np.linspace(phi_2mod-n_points*step_size, phi_2mod+n_points*step_size, 2*n_points+1)

accepted_phi_1mod_perturbed = []
accepted_phi_2mod_perturbed = []
accepted_Fs_mod = []

In [ ]:
ctr = 0

for phi_1m in phi_1mod_perturbed:
    for phi_2m in  phi_2mod_perturbed:
        if phi_1m<0 or phi_2m<0 or phi_1m>1 or phi_2m>1:
            continue
        
        phi_3m = 1 - phi_1m - phi_2m
        if phi_3m<0 or phi_3m>1:
            continue

        # Calculate the concentrations in the 3rd compartment
        phi_13mod = (phi_global[0] - phi_1m*(eta_mod))/(phases.volumes[2])
        phi_23mod = (phi_global[1] - phi_2m*(eta_mod))/(phases.volumes[2])

        if phi_13mod<0 or phi_13mod>1 or phi_23mod<0 or phi_23mod>1:
            continue
        
        phi_33mod = 1 - phi_13mod - phi_23mod
        if phi_33mod<0 or phi_33mod>1:
            continue
        ctr += 1

        phi_in_kmod = [phi_1m, phi_2m, phi_3m]
        phi_in_k3 = [phi_13mod, phi_23mod, phi_33mod]
        
        F = 0
        F = eta_mod*floryHuggins(phi_in_kmod, chis) + phases.volumes[2]*floryHuggins(phi_in_k3, chis)

        accepted_phi_1mod_perturbed.append(phi_1m)
        accepted_phi_2mod_perturbed.append(phi_2m)
        accepted_Fs_mod.append(F)

In [ ]:
ctr/1001**2

In [ ]:
df = pd.DataFrame()
df["phi_1mod"] = accepted_phi_1mod_perturbed
df["phi_2mod"] = accepted_phi_2mod_perturbed
df["F"] = accepted_Fs_mod

df

In [ ]:
np.min(df["F"])

In [ ]:


# Create figure with larger dimensions
fig = go.Figure()

# Main scatter plot with colorbar
fig.add_trace(go.Scatter3d(
    x=pd.Series(df["phi_1mod"].values.ravel()),
    y=pd.Series(df["phi_2mod"].values.ravel()),
    z=pd.Series(df["F"].values.ravel()),
    mode='markers',
    marker=dict(
        size=3,
        color=pd.Series(df["F"].values.ravel()),  # Color by F values
        colorscale='Viridis',
        opacity = 0.1,
        colorbar=dict(
            title='F',  # Colorbar title
            titleside='right',
            titlefont=dict(size=14),
            thickness=20,
            len=0.75,
            x=1.1  # Position adjustment
        ),
        cmin=df["F"].min(),  # Set color scale range
        cmax=df["F"].max()
    ),
    showlegend=False  # Add this to each trace

))

# Model points
fig.add_trace(go.Scatter3d(
    x=pd.Series(phi_1mod.ravel()),
    y=pd.Series(phi_2mod.ravel()),
    z=pd.Series(F_mod.ravel()),
    mode='markers',
    marker=dict(
        size=5,
        color='orange',
        symbol='circle'
    ),
    showlegend=False  # Add this to each trace
))

# Update layout with larger size and other settings
fig.update_layout(
    title={
    'text': "C1+C2",  # Title text
    'y':0.95,  # Position from top (0-1)
    'x':0.35,   # Center the title
    'xanchor': 'center',
    'yanchor': 'top',
    'font': dict(
        size=20,  # Title font size
        color='black'  # Title color
    )
    },

    width=1200,  # Width in pixels
    height=800,   # Height in pixels
    autosize=False,  # Disable autosizing
    scene=dict(
        xaxis_title='phi_1mod',
        yaxis_title='phi_2mod',
        zaxis_title='F_sys',
        aspectmode='manual',  # Preserve aspect ratio
        aspectratio=dict(x=1, y=1, z=0.7)  # Adjust these values as needed
    ),
    margin=dict(l=0, r=500, b=0, t=30)  # Increased right margin for colorbar
)

fig.show()

In [ ]:
min_F_row = df[df['F'] == df['F'].min()]
min_F_row

In [ ]:
min_F_row["phi_1mod"].values[0], min_F_row["phi_2mod"].values[0], 1-min_F_row["phi_2mod"].values[0]-min_F_row["phi_1mod"].values[0]

In [ ]:
phi_1mod, phi_2mod, 1-phi_1mod-phi_2mod


In [ ]:
(1-phi_1mod-phi_2mod)/(1-min_F_row["phi_2mod"].values[0]-min_F_row["phi_1mod"].values[0])

#### Merging compartments 1 and 3

In [ ]:
# Interaction matrix
chis = np.array([[0, 3.0, 3.0], [3.0, 0.0, 3.0], [3.0, 3.0, 0.0]], dtype = DTYPE)

# Global value of phis
phi_global = np.array([0.4, 0.3, 0.3], dtype = DTYPE)

# Equilibrium 3 phase solution
phases = flory.find_coexisting_phases(3, chis, phi_global, progress=False)

print("\n" + f"Equlibrium phase volumes: {phases.volumes}")
print("Equlibrium phase concentrations:\n" + f"{phases.fractions}")
phases.fractions = np.transpose(phases.fractions)


In [ ]:
# Merging compartments 1 and 3

# First, calculate the modified compositions in the hybrid compartment (1+3)
eta_mod = phases.volumes[0] + phases.volumes[2]
phi_1mod = (phases.volumes[0]*phases.fractions[0, 0] + phases.volumes[2]*phases.fractions[0, 2])/eta_mod
phi_2mod = (phases.volumes[0]*phases.fractions[1, 0] + phases.volumes[2]*phases.fractions[1, 2])/eta_mod
phi_3mod = 1 - phi_1mod - phi_2mod

# So, now we have two phases with [phi_1mod, phi_2mod, phi_3mod] and [phases.fractions[0, 1], phases.fractions[1, 21, phases.fractions[2, 1]]
# with volumes (phases.volumes[0] + phases.volumes[2]) and phases.volumes[1] respectively

# For perturbations, we keep the volumes of these individual phases fixed
# Essentially, only perturb the concentrations

phi_in_kmod = [phi_1mod, phi_2mod, phi_3mod]
phi_in_k2 = [phases.fractions[0, 1], phases.fractions[1, 1], phases.fractions[2, 1]]

# Calculate the Free Energy of this modified system
F_mod = eta_mod*floryHuggins(phi_in_kmod, chis) + phases.volumes[1]*floryHuggins(phi_in_k2, chis)
print(F_mod)

In [ ]:
# Effectively 2 degrees of freedom in the system
step_size = DTYPE(0.001)
n_points = 1000

phi_1mod_perturbed = np.linspace(phi_1mod-n_points*step_size, phi_1mod+n_points*step_size, 2*n_points+1)
phi_2mod_perturbed = np.linspace(phi_2mod-n_points*step_size, phi_2mod+n_points*step_size, 2*n_points+1)

accepted_phi_1mod_perturbed = []
accepted_phi_2mod_perturbed = []
accepted_Fs_mod = []

In [ ]:
ctr = 0

for phi_1m in phi_1mod_perturbed:
    for phi_2m in  phi_2mod_perturbed:
        if phi_1m<0 or phi_2m<0 or phi_1m>1 or phi_2m>1:
            continue
        
        phi_3m = 1 - phi_1m - phi_2m
        if phi_3m<0 or phi_3m>1:
            continue

        # Calculate the concentrations in the 2nd compartment
        phi_12mod = (phi_global[0] - phi_1m*(eta_mod))/(phases.volumes[1])
        phi_22mod = (phi_global[1] - phi_2m*(eta_mod))/(phases.volumes[1])

        if phi_12mod<0 or phi_12mod>1 or phi_22mod<0 or phi_22mod>1:
            continue
        
        phi_32mod = 1 - phi_12mod - phi_22mod
        if phi_32mod<0 or phi_32mod>1:
            continue
        ctr += 1

        phi_in_kmod = [phi_1m, phi_2m, phi_3m]
        phi_in_k2 = [phi_12mod, phi_22mod, phi_32mod]
        
        F = 0
        F = eta_mod*floryHuggins(phi_in_kmod, chis) + phases.volumes[1]*floryHuggins(phi_in_k2, chis)

        accepted_phi_1mod_perturbed.append(phi_1m)
        accepted_phi_2mod_perturbed.append(phi_2m)
        accepted_Fs_mod.append(F)

In [ ]:
df = pd.DataFrame()
df["phi_1mod"] = accepted_phi_1mod_perturbed
df["phi_2mod"] = accepted_phi_2mod_perturbed
df["F"] = accepted_Fs_mod

df

In [ ]:
np.min(df["F"])

In [ ]:


# Create figure with larger dimensions
fig = go.Figure()

# Main scatter plot with colorbar
fig.add_trace(go.Scatter3d(
    x=pd.Series(df["phi_1mod"].values.ravel()),
    y=pd.Series(df["phi_2mod"].values.ravel()),
    z=pd.Series(df["F"].values.ravel()),
    mode='markers',
    marker=dict(
        size=3,
        color=pd.Series(df["F"].values.ravel()),  # Color by F values
        colorscale='Viridis',
        opacity = 0.1,
        colorbar=dict(
            title='F',  # Colorbar title
            titleside='right',
            titlefont=dict(size=14),
            thickness=20,
            len=0.75,
            x=1.1  # Position adjustment
        ),
        cmin=df["F"].min(),  # Set color scale range
        cmax=df["F"].max()
    ),
    showlegend=False  # Add this to each trace

))

# Model points
fig.add_trace(go.Scatter3d(
    x=pd.Series(phi_1mod.ravel()),
    y=pd.Series(phi_2mod.ravel()),
    z=pd.Series(F_mod.ravel()),
    mode='markers',
    marker=dict(
        size=5,
        color='orange',
        symbol='circle'
    ),
    showlegend=False  # Add this to each trace
))

# Update layout with larger size and other settings
fig.update_layout(
    title={
    'text': "C1+C3",  # Title text
    'y':0.95,  # Position from top (0-1)
    'x':0.35,   # Center the title
    'xanchor': 'center',
    'yanchor': 'top',
    'font': dict(
        size=20,  # Title font size
        color='black'  # Title color
    )
    },

    width=1200,  # Width in pixels
    height=800,   # Height in pixels
    autosize=False,  # Disable autosizing
    scene=dict(
        xaxis_title='phi_1mod',
        yaxis_title='phi_2mod',
        zaxis_title='F_sys',
        aspectmode='manual',  # Preserve aspect ratio
        aspectratio=dict(x=1, y=1, z=0.7)  # Adjust these values as needed
    ),
    margin=dict(l=0, r=500, b=0, t=30)  # Increased right margin for colorbar
)

fig.show()

#### Merging compartments 2 and 3

In [ ]:
# Interaction matrix
chis = np.array([[0, 3.0, 3.0], [3.0, 0.0, 3.0], [3.0, 3.0, 0.0]], dtype = DTYPE)

# Global value of phis
phi_global = np.array([0.4, 0.3, 0.3], dtype = DTYPE)

# Equilibrium 3 phase solution
phases = flory.find_coexisting_phases(3, chis, phi_global, progress=False)

print("\n" + f"Equlibrium phase volumes: {phases.volumes}")
print("Equlibrium phase concentrations:\n" + f"{phases.fractions}")
phases.fractions = np.transpose(phases.fractions)


In [ ]:
# Merging compartments 2 and 3

# First, calculate the modified compositions in the hybrid compartment (1+3)
eta_mod = phases.volumes[1] + phases.volumes[2]
phi_1mod = (phases.volumes[1]*phases.fractions[0, 1] + phases.volumes[2]*phases.fractions[0, 2])/eta_mod
phi_2mod = (phases.volumes[1]*phases.fractions[1, 1] + phases.volumes[2]*phases.fractions[1, 2])/eta_mod
phi_3mod = 1 - phi_1mod - phi_2mod

# So, now we have two phases with [phi_1mod, phi_2mod, phi_3mod] and [phases.fractions[0, 1], phases.fractions[1, 21, phases.fractions[2, 1]]
# with volumes (phases.volumes[0] + phases.volumes[2]) and phases.volumes[1] respectively

# For perturbations, we keep the volumes of these individual phases fixed
# Essentially, only perturb the concentrations

phi_in_kmod = [phi_1mod, phi_2mod, phi_3mod]
phi_in_k1 = [phases.fractions[0, 0], phases.fractions[1, 0], phases.fractions[2, 0]]

# Calculate the Free Energy of this modified system
F_mod = eta_mod*floryHuggins(phi_in_kmod, chis) + phases.volumes[0]*floryHuggins(phi_in_k1, chis)
print(F_mod)

In [ ]:
# Effectively 2 degrees of freedom in the system
step_size = DTYPE(0.001)
n_points = 1000

phi_1mod_perturbed = np.linspace(phi_1mod-n_points*step_size, phi_1mod+n_points*step_size, 2*n_points+1)
phi_2mod_perturbed = np.linspace(phi_2mod-n_points*step_size, phi_2mod+n_points*step_size, 2*n_points+1)

accepted_phi_1mod_perturbed = []
accepted_phi_2mod_perturbed = []
accepted_Fs_mod = []

In [ ]:
ctr = 0

for phi_1m in phi_1mod_perturbed:
    for phi_2m in  phi_2mod_perturbed:
        if phi_1m<0 or phi_2m<0 or phi_1m>1 or phi_2m>1:
            continue
        
        phi_3m = 1 - phi_1m - phi_2m
        if phi_3m<0 or phi_3m>1:
            continue

        # Calculate the concentrations in the 1st compartment
        phi_11mod = (phi_global[0] - phi_1m*(eta_mod))/(phases.volumes[0])
        phi_21mod = (phi_global[1] - phi_2m*(eta_mod))/(phases.volumes[0])

        if phi_11mod<0 or phi_11mod>1 or phi_21mod<0 or phi_21mod>1:
            continue
        
        phi_31mod = 1 - phi_11mod - phi_21mod
        if phi_31mod<0 or phi_31mod>1:
            continue
        ctr += 1

        phi_in_kmod = [phi_1m, phi_2m, phi_3m]
        phi_in_k1 = [phi_11mod, phi_21mod, phi_31mod]
        
        F = 0
        F = eta_mod*floryHuggins(phi_in_kmod, chis) + phases.volumes[0]*floryHuggins(phi_in_k1, chis)

        accepted_phi_1mod_perturbed.append(phi_1m)
        accepted_phi_2mod_perturbed.append(phi_2m)
        accepted_Fs_mod.append(F)

In [ ]:
df = pd.DataFrame()
df["phi_1mod"] = accepted_phi_1mod_perturbed
df["phi_2mod"] = accepted_phi_2mod_perturbed
df["F"] = accepted_Fs_mod

df

In [ ]:
np.min(df["F"])

In [ ]:


# Create figure with larger dimensions
fig = go.Figure()

# Main scatter plot with colorbar
fig.add_trace(go.Scatter3d(
    x=pd.Series(df["phi_1mod"].values.ravel()),
    y=pd.Series(df["phi_2mod"].values.ravel()),
    z=pd.Series(df["F"].values.ravel()),
    mode='markers',
    marker=dict(
        size=3,
        color=pd.Series(df["F"].values.ravel()),  # Color by F values
        colorscale='Viridis',
        opacity = 0.1,
        colorbar=dict(
            title='F',  # Colorbar title
            titleside='right',
            titlefont=dict(size=14),
            thickness=20,
            len=0.75,
            x=1.1  # Position adjustment
        ),
        cmin=df["F"].min(),  # Set color scale range
        cmax=df["F"].max()
    ),
    showlegend=False  # Add this to each trace

))

# Model points
fig.add_trace(go.Scatter3d(
    x=pd.Series(phi_1mod.ravel()),
    y=pd.Series(phi_2mod.ravel()),
    z=pd.Series(F_mod.ravel()),
    mode='markers',
    marker=dict(
        size=5,
        color='orange',
        symbol='circle'
    ),
    showlegend=False  # Add this to each trace
))

# Update layout with larger size and other settings
fig.update_layout(
    title={
    'text': "C2+C3",  # Title text
    'y':0.95,  # Position from top (0-1)
    'x':0.35,   # Center the title
    'xanchor': 'center',
    'yanchor': 'top',
    'font': dict(
        size=20,  # Title font size
        color='black'  # Title color
    )
    },

    width=1200,  # Width in pixels
    height=800,   # Height in pixels
    autosize=False,  # Disable autosizing
    scene=dict(
        xaxis_title='phi_1mod',
        yaxis_title='phi_2mod',
        zaxis_title='F_sys',
        aspectmode='manual',  # Preserve aspect ratio
        aspectratio=dict(x=1, y=1, z=0.7)  # Adjust these values as needed
    ),
    margin=dict(l=0, r=500, b=0, t=30)  # Increased right margin for colorbar
)

fig.show()

In [ ]:
ctr = 0
for  phi_1m in phi_1mod_perturbed:
    for phi_2m in phi_2mod_perturbed:
        for phi_3m in phi_3mod_perturbed:
            if phi_1m<0 or phi_2m<0 or phi_3m<0 or phi_1m>1 or phi_2m>1 or phi_3m>1:
                continue
                
            phi_in_kmod = [phi_1m, phi_2m, phi_3m]
            # print(np.isclose(np.sum(phi_in_kmod), 1, atol =1e-7))
            if np.isclose(np.sum(phi_in_kmod), 1, atol =1e-7):
                ctr += 1
                    
                F = 0
                F = eta_mod*floryHuggins(phi_in_kmod, chis) + phases.volumes[1]*floryHuggins(phi_in_k2, chis)
                
                accepted_phi_1mod_perturbed.append(phi_1m)
                accepted_phi_2mod_perturbed.append(phi_2m)
                accepted_phi_3mod_perturbed.append(phi_3m)
                accepted_Fs_mod.append(F)

In [ ]:
ctr/21**3

In [ ]:
df = pd.DataFrame()
df["phi_1mod"] = accepted_phi_1mod_perturbed
df["phi_2mod"] = accepted_phi_1mod_perturbed
df["phi_3mod"] = accepted_phi_2mod_perturbed
df["F"] = accepted_Fs_mod

df

In [ ]:
min_F_row = df[df['F'] == df['F'].min()]
min_F_row

In [ ]:
min_F_row["phi_1mod"].values[0]+min_F_row["phi_2mod"].values[0]+min_F_row["phi_3mod"].values[0]


In [ ]:
sum_condition = (df['phi_1mod'] + df['phi_2mod'] + df['phi_3mod']).round(3) == 1.0
filtered_df = df[sum_condition]
filtered_df

In [ ]:
min_F_row = filtered_df[filtered_df['F'] == filtered_df['F'].min()]
min_F_row

In [ ]:
phases.fractions

In [ ]:
F_flory

In [ ]:
F_perturb

In [ ]:
eta_1_perturbed = np.linspace(phases.volumes[0] - n_points*step_size, phases.volumes[0]+n_points*step_size, 2*n_points+1)
eta_2_perturbed = np.linspace(phases.volumes[0] - n_points*step_size, phases.volumes[0]+n_points*step_size, 2*n_points+1)
phi_11_perturbed = np.linspace(phases.fractions[0, 0] - n_points*step_size, phases.fractions[0, 0]+n_points*step_size, 2*n_points+1)
phi_12_perturbed = np.linspace(phases.fractions[0, 1] - n_points*step_size, phases.fractions[0, 1]+n_points*step_size, 2*n_points+1)
phi_21_perturbed = np.linspace(phases.fractions[1, 0] - n_points*step_size, phases.fractions[1, 0]+n_points*step_size, 2*n_points+1)
phi_22_perturbed = np.linspace(phases.fractions[1, 1] - n_points*step_size, phases.fractions[1, 1]+n_points*step_size, 2*n_points+1)


In [ ]:
len(eta_1_perturbed)**6

In [ ]:
accepted_eta_1 = []
accepted_eta_2 = []
accepted_phi_11 = []
accepted_phi_12 = []
accepted_phi_21 = []
accepted_phi_22 = []
accepted_F = []

In [ ]:
for eta_1 in tqdm(eta_1_perturbed):
    for eta_2 in eta_2_perturbed:
        if eta_1 < 0 or eta_1 > 1 or eta_2 < 0 or eta_2 > 1:
            continue  # Skip invalid eta values
        eta_3 = 1 - eta_1 - eta_2
        if eta_3 < 0 or eta_3 > 1:
            continue  # Skip invalid eta_3 values

        for phi_11 in phi_11_perturbed:
            for phi_12 in phi_12_perturbed:
                if phi_11 < 0 or phi_11 > 1 or phi_12 < 0 or phi_12 > 1:
                    continue  # Skip invalid phi values
                phi_13 = 1 - phi_11 - phi_12
                if phi_13 < 0 or phi_13 > 1:
                    continue  # Skip invalid phi_13 values

                for phi_21 in phi_21_perturbed:
                    for phi_22 in phi_22_perturbed:
                        if phi_21 < 0 or phi_21 > 1 or phi_22 < 0 or phi_22 > 1:
                            continue  # Skip invalid phi values
                        phi_23 = 1 - phi_21 - phi_22
                        if phi_23 < 0 or phi_23 > 1:
                            continue  # Skip invalid phi_23 values

                        # Calculate phi_13 and phi_23
                        phi_13 = (phi_global[0] - (eta_1 * phi_11 + eta_2 * phi_21)) / eta_3
                        phi_23 = (phi_global[1] - (eta_1 * phi_12 + eta_2 * phi_22)) / eta_3

                        if phi_13 < 0 or phi_13 > 1 or phi_23 < 0 or phi_23 > 1:
                            continue  # Skip invalid phi values

                        # Calculate phi_31, phi_32, and phi_33
                        phi_31 = 1 - phi_11 - phi_12
                        phi_32 = 1 - phi_21 - phi_22
                        phi_33 = 1 - phi_13 - phi_23

                        # Define phase compositions
                        phi_in_k1 = [phi_11, phi_21, phi_31]
                        phi_in_k2 = [phi_12, phi_22, phi_32]
                        phi_in_k3 = [phi_13, phi_23, phi_33]

                        # Calculate free energy
                        F = 0
                        F += eta_1 * floryHuggins(phi_in_k1, chis)
                        F += eta_2 * floryHuggins(phi_in_k2, chis)
                        F += eta_3 * floryHuggins(phi_in_k3, chis)


                        accepted_eta_1.append(eta_1)
                        accepted_eta_2.append(eta_2)

                        accepted_phi_11.append(phi_11)
                        accepted_phi_12.append(phi_12)
                        accepted_phi_21.append(phi_21)
                        accepted_phi_22.append(phi_22)
                        
                        accepted_F.append(F)

In [ ]:
df = pd.DataFrame()
df["eta_1"] = accepted_eta_1
df["eta_2"] = accepted_eta_2
df["phi_11"] = accepted_phi_11
df["phi_12"] = accepted_phi_12
df["phi_21"] = accepted_phi_21
df["phi_22"] = accepted_phi_22
df["F"] = accepted_F

In [ ]:
df

In [ ]:
len(df)/(len(eta_1_perturbed)**6)

In [ ]:
# Calculating the FH free energy of the phase-separated state

# f1 = floryHuggins(phases.fractions[0, :], chis)
# f2 = floryHuggins(phases.fractions[1, :], chis)
# f3 = floryHuggins(phases.fractions[2, :], chis)

F = 0
for i in range(len(phases.volumes)):
    F += phases.volumes[i]*floryHuggins(phases.fractions[i, :], chis)

F

In [ ]:
np.min(df["F"])

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")

# Plot the data
ax.scatter(df["eta_1"], df["phi_11"], df["F"], c=df["F"], cmap="viridis", marker="o", alpha = 0.008, s = 1)
ax.scatter(phases.volumes[0], phases.fractions[0,0],  F, color = "orange")


In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")

# Plot the data
ax.scatter(df["phi_11"], df["phi_12"], df["F"], c=df["F"], cmap="viridis", marker="o", alpha = 0.008)
ax.scatter(phases.fractions[0,0], phases.fractions[0,1],  F, color = "orange")

cbar = plt.colorbar(ax.scatter(df["phi_11"], df["phi_12"], df["F"], c=df["F"], cmap="viridis"))
cbar.set_label("Free Energy (F)")

In [ ]:
type(df["F"].values[0])